In [1]:
import json
import re
import urllib
from pprint import pprint
import time
from tqdm import tqdm

from py2neo import Node, Graph, Relationship, NodeMatcher
from py2neo.bulk import merge_nodes

import numpy as np
import pandas as pd
import wikipedia
from sklearn.metrics.pairwise import cosine_similarity

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.matcher import Matcher
from spacy.tokens import Doc, Span, Token

print(spacy.__version__)

3.0.3


# Configure spacy

Prior to actually using spacy, we need to load in some models.  The basic model is their small core library, taken from the web: `en_core_web_sm`, which provides good, basic functionality with a small download size (< 20 MB).  However, one drawback of this basic model is that it doesn't have full word vectors.  Instead, it comes with context-sensitive tensors.  You can still do things like text similarity with it, but if you want to use spacy to create good word vectors, you should use a larger model such as `en_core_web_md` or`en_core_web_lg` since the small models are not known for accuracy.  You can also use a variety of third-party models, but that is beyond the scope of this workshop.  Again, choose the model that works best with your setup.

In general, to load the models we use the following command:

`python3 -m spacy download en_core_web_md`

This has already been completed in the container, but if you want to add other models you can do this either as a cell in this notebook or via the CLI.

## API key for Google Knowledge Graph

See below for instructions on how to create this key.  When you have the key, save it to a file called `.api_key` in this directory.

In [2]:
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
VERBS = ['ROOT', 'advcl']
OBJECTS = ["dobj", "dative", "attr", "oprd", 'pobj']
ENTITY_LABELS = ['PERSON', 'NORP', 'GPE', 'ORG', 'FAC', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART']

api_key = open('.api_key').read()

non_nc = spacy.load('en_core_web_md')

nlp = spacy.load('en_core_web_md')
nlp.add_pipe('merge_noun_chunks')

print(non_nc.pipe_names)
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']
['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer', 'merge_noun_chunks']


# Query Google Knowledge Graph

To query the Google Knowledge Graph you will require an API key, which permits you to have 100,000 read calls per day per project for free.  That will be more than sufficient for this workshop.  To obtain your key, follow [these instructions](https://developers.google.com/knowledge-graph/how-tos/authorizing).

In [3]:
def query_google(query, api_key, limit=10, indent=True, return_lists=True):
    
    text_ls = []
    node_label_ls = []
    url_ls = []
    
    params = {
        'query': query,
        'limit': limit,
        'indent': indent,
        'key': api_key,
    }   
    
    service_url = 'https://kgsearch.googleapis.com/v1/entities:search'
    url = service_url + '?' + urllib.parse.urlencode(params)
    response = json.loads(urllib.request.urlopen(url).read())
    
    if return_lists:
        for element in response['itemListElement']:

            try:
                node_label_ls.append(element['result']['@type'])
            except:
                node_label_ls.append('')

            try:
                text_ls.append(element['result']['detailedDescription']['articleBody'])
            except:
                text_ls.append('')
                
            try:
                url_ls.append(element['result']['detailedDescription']['url'])
            except:
                url_ls.append('')
                
        return text_ls, node_label_ls, url_ls
    
    else:
        return response

# NLP Functions

There are a variety of functions below that will use both spacy and regex to clean and prepare our data prior to loading it into the graph database.  The main goal is to create subject-verb-object (SVO) triples.  But before we can do that, we have to do some general cleaning of the data.  The below functions perform this for us in the following order:

1. Use regex to remove control characters (`remove_special_characters`)
2. Create spacy doc of (1) (`create_svo_lists`)
3. Get list of all subjects, verbs, and objects (`create_svo_lists`)
4. For each object, find the closest verb (`create_svo_triples`)
5. Remove all stop words and punctuation (`remove_stop_words_and_punct`)
6. Assemble SVO tuples (`create_svo_triples`)
7. Remove duplicate tuples (`remove_duplicates`)
8. Remove tuples that have dates in them (`remove_dates`)

In [4]:
def remove_special_characters(text):
    
    regex = re.compile(r'[\n\r\t]')
    clean_text = regex.sub(" ", text)
    
    return clean_text


def remove_stop_words_and_punct(text, print_text=False):
    
    result_ls = []
    rsw_doc = non_nc(text)
    
    for token in rsw_doc:
        if print_text:
            print(token, token.is_stop)
            print('--------------')
        if not token.is_stop and not token.is_punct:
            result_ls.append(str(token))
    
    result_str = ' '.join(result_ls)

    return result_str


def create_svo_lists(doc, print_lists=False):
    
    subject_ls = []
    verb_ls = []
    object_ls = []

    for token in doc:
        if token.dep_ in SUBJECTS:
            subject_ls.append((token.lower_, token.idx))
        elif token.dep_ in VERBS:
            verb_ls.append((token.lemma_, token.idx))
        elif token.dep_ in OBJECTS:
            object_ls.append((token.lower_, token.idx))

    if print_lists:
        print('SUBJECTS: ', subject_ls)
        print('VERBS: ', verb_ls)
        print('OBJECTS: ', object_ls)
    
    return subject_ls, verb_ls, object_ls


def remove_duplicates(tup, tup_posn):
    
    check_val = set()
    result = []
    
    for i in tup:
        if i[tup_posn] not in check_val:
            result.append(i)
            check_val.add(i[tup_posn])
            
    return result


def remove_dates(tup_ls):
    
    clean_tup_ls = []
    for entry in tup_ls:
        if not entry[2].isdigit():
            clean_tup_ls.append(entry)
    return clean_tup_ls


def create_svo_triples(text):
    
    clean_text = remove_special_characters(text)
    doc = nlp(clean_text)
    subject_ls, verb_ls, object_ls = create_svo_lists(doc)
    
    graph_tup_ls = []
    dedup_tup_ls = []
    clean_tup_ls = []
    
    for subj in subject_ls: 
        for obj in object_ls:
            
            dist_ls = []
            
            for v in verb_ls:
                
                # Assemble a list of distances between each object and each verb
                dist_ls.append(abs(obj[1] - v[1]))
                
            # Get the index of the verb with the smallest distance to the object 
            # and return that verb
            index_min = min(range(len(dist_ls)), key=dist_ls.__getitem__)
            
            # Remve stop words from subjects and object.  Note that we do this a bit
            # later down in the process to allow for proper sentence recognition.

            no_sw_subj = remove_stop_words_and_punct(subj[0])
            no_sw_obj = remove_stop_words_and_punct(obj[0])
            
            # Add entries to the graph iff neither subject nor object is blank
            if no_sw_subj and no_sw_obj:
                tup = (no_sw_subj, verb_ls[index_min][0], no_sw_obj)
                graph_tup_ls.append(tup)
        
        #clean_tup_ls = remove_dates(graph_tup_ls)
    
    dedup_tup_ls = remove_duplicates(graph_tup_ls, 2)
    clean_tup_ls = remove_dates(dedup_tup_ls)
    
    return clean_tup_ls

# Let's get started!

In [5]:
text = wikipedia.summary('barack obama')
text

'Barack Hussein Obama II ( (listen) bə-RAHK hoo-SAYN oh-BAH-mə; born August 4, 1961) is an American politician and attorney who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, Obama was the first African-American president of the United States. He previously served as a U.S. senator from Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.\nObama was born in Honolulu, Hawaii. After graduating from Columbia University in 1983, he worked as a community organizer in Chicago. In 1988, he enrolled in Harvard Law School, where he was the first black person to be president of the Harvard Law Review. After graduating, he became a civil rights attorney and an academic, teaching constitutional law at the University of Chicago Law School from 1992 to 2004. Turning to elective politics, he represented the 13th district from 1997 until 2004 in the Illinois Senate, when he ran for the U.S. Senate. Obama received nationa

# Preprocess the data (including NLP)

We have obtained our text from Wikipedia.  Now we will prepare it for the graph.  This will be done in the following steps:

1. We create the SVO triples for the initial test (`create_svo_triples`)
2. Get some node properties for each object such as the different possible label types, a brief description, and the URL provided by the Google Knowledge Graph (`get_obj_properties`)
3. Add a new layer to the graph that is the SVOs associated with each object.  This is done to build out the graph a bit and make it more rich. (`add_layer`)
4. Combine the initial tuple list from (1) with the expanded list from (3)
5. Eliminate tuples where the subject is equal to the object (`subj_equals_obj`)
6. Create word embeddings of the object descriptions (`create_word_vectors`)

In [29]:
def get_obj_properties(tup_ls):
    
    init_obj_tup_ls = []
    
    for tup in tup_ls:

        try:
            text, node_label_ls, url = query_google(tup[2], api_key, limit=1)
            new_tup = (tup[0], tup[1], tup[2], text[0], node_label_ls[0], url[0])
        except:
            new_tup = (tup[0], tup[1], tup[2], [], [], [])
        
        init_obj_tup_ls.append(new_tup)
        
    return init_obj_tup_ls


def add_layer(tup_ls):

    svo_tup_ls = []
    
    for tup in tup_ls:
        
        if tup[3]:
            svo_tup = create_svo_triples(tup[3])
            svo_tup_ls.extend(svo_tup)
        else:
            continue
    
    return get_obj_properties(svo_tup_ls)
        

def subj_equals_obj(tup_ls):
    
    new_tup_ls = []
    
    for tup in tup_ls:
        if tup[0] != tup[2]:
            new_tup_ls.append((tup[0], tup[1], tup[2], tup[3], tup[4], tup[5]))
            
    return new_tup_ls


def check_for_string_labels(tup_ls):
    # This is for an edge case where the object does not get fully populated
    # resulting in the node labels being assigned to string instead of list
    
    clean_tup_ls = []
    
    for el in tup_ls:
        if isinstance(el[2], list):
            clean_tup_ls.append(el)
            
    return clean_tup_ls


def create_word_vectors(tup_ls):

    new_tup_ls = []
    
    for tup in tup_ls:
        if tup[3]:
            doc = nlp(tup[3])
            new_tup = (tup[0], tup[1], tup[2], tup[3], tup[4], tup[5], doc.vector)
        else:
            new_tup = (tup[0], tup[1], tup[2], tup[3], tup[4], tup[5], np.random.uniform(low=-1.0, high=1.0, size=(300,)))
        new_tup_ls.append(new_tup)
        
    return new_tup_ls

# Let's now run this step by step...

In [7]:
%%time
initial_tup_ls = create_svo_triples(text)

CPU times: user 49 s, sys: 38.3 ms, total: 49.1 s
Wall time: 49.1 s


In [8]:
%%time
init_obj_tup_ls = get_obj_properties(initial_tup_ls)
new_layer_ls = add_layer(init_obj_tup_ls)
starter_edge_ls = init_obj_tup_ls + new_layer_ls
edge_ls = subj_equals_obj(starter_edge_ls)
clean_edge_ls = check_for_string_labels(edge_ls)
clean_edge_ls[0:3]

CPU times: user 23 s, sys: 469 ms, total: 23.5 s
Wall time: 1min 33s


[]

In [9]:
%%time
edges_word_vec_ls = create_word_vectors(edge_ls)

CPU times: user 4.11 s, sys: 0 ns, total: 4.11 s
Wall time: 4.11 s


# Create the node and edge lists to populate the graph with the below helper functions

These functions achieve the following:

1. Deduping of the node list (`dedup`)
2. Under the idea that we might want to use word embeddings, we are pulling them in as a node property.  However, Neo4j doesn't work well with numpy arrays, so we convert that array to a list of floats (`convert_vec_to_ls`).
3. Add nodes to the graph with the `py2neo` bulk importer (`add_nodes`)
4. Add edges (relationships) to the graph (`add_edges`)

In [10]:
def dedup(tup_ls):
    
    visited = set()
    output_ls = []
    
    for tup in tup_ls:
        if not tup[0] in visited:
            visited.add(tup[0])
            output_ls.append((tup[0], tup[1], tup[2], tup[3], tup[4]))
            
    return output_ls


def convert_vec_to_ls(tup_ls):
    
    vec_to_ls_tup = []
    
    for el in tup_ls:
        vec_ls = [float(v) for v in el[4]]
        tup = (el[0], el[1], el[2], el[3], vec_ls)
        vec_to_ls_tup.append(tup)
        
    return vec_to_ls_tup


def add_nodes(tup_ls):   

    keys = ['name', 'description', 'node_labels', 'url', 'word_vec']
    merge_nodes(graph.auto(), tup_ls, ('Node', 'name'), keys=keys)
    print('Number of nodes in graph: ', graph.nodes.match('Node').count())
    
    return


def add_edges(edge_ls):
    
    edge_dc = {} 
    
    # Group tuple by verb
    # Result: {verb1: [(sub1, v1, obj1), (sub2, v2, obj2), ...],
    #          verb2: [(sub3, v3, obj3), (sub4, v4, obj4), ...]}
    
    for tup in edge_ls: 
        if tup[1] in edge_dc: 
            edge_dc[tup[1]].append((tup[0], tup[1], tup[2])) 
        else: 
            edge_dc[tup[1]] = [(tup[0], tup[1], tup[2])] 
    
    for edge_labels, tup_ls in tqdm(edge_dc.items()):   # k=edge labels, v = list of tuples
        
        tx = graph.begin()
        
        for el in tup_ls:
            source_node = nodes_matcher.match(name=el[0]).first()
            target_node = nodes_matcher.match(name=el[2]).first()
            if not source_node:
                source_node = Node('Node', name=el[0])
                tx.create(source_node)
            if not target_node:
                try:
                    target_node = Node('Node', name=el[2], node_labels=el[4], url=el[5], word_vec=el[6])
                    tx.create(target_node)
                except:
                    continue
            try:
                rel = Relationship(source_node, edge_labels, target_node)
            except:
                continue
            tx.create(rel)
        tx.commit()
    
    return

# Creating some lists of tuples representing the node and edge lists

For our node list, we begin with the query subject (Barack Obama), which is in `edge_ls[0][0]` and put this in the variable `orig_node_tup_ls`.  We assume a node label of `Subject` and no description or word vector.  We then add all of the objects associated with the `edges_word_vec_ls` (`obj_node_tup_ls`) and combine that with the previous variable to create `full_node_tup_ls`, which we then dedupe into `dedup_node_tup_ls`.

In [11]:
orig_node_tup_ls = [(edge_ls[0][0], '', ['Subject'], '', np.random.uniform(low=-1.0, high=1.0, size=(300,)))]
obj_node_tup_ls = [(tup[2], tup[3], tup[4], tup[5], tup[6]) for tup in edges_word_vec_ls]
full_node_tup_ls = orig_node_tup_ls + obj_node_tup_ls
dedup_node_tup_ls = dedup(full_node_tup_ls)

In [12]:
len(full_node_tup_ls), len(dedup_node_tup_ls)

(657, 507)

# Create the node list that will be used to populate the graph...

In [13]:
node_tup_ls = convert_vec_to_ls(dedup_node_tup_ls)

# Populate the graph

Here we establish a connection to the database, which is running on the internal Docker network called `neo4j`.  We provide it the user name and password and also create a class for matching nodes (used when we establish the edges in the graph).  Finally, we add the nodes and edges to populate the database.

In [15]:
graph = Graph("bolt://neo4j:7687", name="neo4j", password="1234")
nodes_matcher = NodeMatcher(graph)

In [16]:
add_nodes(node_tup_ls)

Number of nodes in graph:  507


In [17]:
add_edges(edges_word_vec_ls)

100%|██████████| 62/62 [00:07<00:00,  7.92it/s]


# But we have some things to explore in the DB before we think we are done with populating the graph...  

Note: I am going to collapse a lot of the tuple list creation into a few functions to make it "easier" on the eyes.

We are going to delete all entries in the database and start it fresh.  The reason for this is that the nodes in there already have labels.  We are going to run our Cypher query to create the new labels, but it would be complicated to do that selectively based just on the new nodes added to the graph after our original EDA.

In [30]:
def edge_tuple_creation(text):
    
    initial_tup_ls = create_svo_triples(text)
    init_obj_tup_ls = get_obj_properties(initial_tup_ls)
    new_layer_ls = add_layer(init_obj_tup_ls)
    starter_edge_ls = init_obj_tup_ls + new_layer_ls
    edge_ls = subj_equals_obj(starter_edge_ls)
    edges_word_vec_ls = create_word_vectors(edge_ls)
    
    return edges_word_vec_ls


def node_tuple_creation(edges_word_vec_ls):
    
    orig_node_tup_ls = [(edges_word_vec_ls[0][0], '', ['Subject'], '', np.random.uniform(low=-1.0, high=1.0, size=(300,)))]
    obj_node_tup_ls = [(tup[2], tup[3], tup[4], tup[5], tup[6]) for tup in edges_word_vec_ls]
    full_node_tup_ls = orig_node_tup_ls + obj_node_tup_ls
    cleaned_node_tup_ls = check_for_string_labels(full_node_tup_ls)
    dedup_node_tup_ls = dedup(cleaned_node_tup_ls)
    node_tup_ls = convert_vec_to_ls(dedup_node_tup_ls)
    
    return node_tup_ls    

In [31]:
%%time
barack_text = wikipedia.summary('barack obama')
barack_edges_word_vec_ls = edge_tuple_creation(barack_text)
barack_node_tup_ls = node_tuple_creation(barack_edges_word_vec_ls)

michelle_text = wikipedia.summary('michelle obama')
michelle_edges_word_vec_ls = edge_tuple_creation(michelle_text)
michelle_node_tup_ls = node_tuple_creation(michelle_edges_word_vec_ls)

CPU times: user 1min 29s, sys: 739 ms, total: 1min 30s
Wall time: 3min 4s


In [32]:
full_node_ls = barack_node_tup_ls + michelle_node_tup_ls
full_edge_ls = barack_edges_word_vec_ls + michelle_edges_word_vec_ls
full_dedup_node_tup_ls = dedup(full_node_ls)
print(len(full_node_ls), len(full_dedup_node_tup_ls))

660 624


In [33]:
add_nodes(full_dedup_node_tup_ls)
add_edges(full_edge_ls)

  0%|          | 0/77 [00:00<?, ?it/s]

Number of nodes in graph:  624


100%|██████████| 77/77 [00:06<00:00, 11.21it/s]


# Entity disambiguation

In this notebook we will calculate the cosine similarity of the word vectors of target nodes to try and determine the likelihood of two nodes being the same entity.  Note that you could do this directly with spacy (which defaults to cosine similarity) prior to anything above through something like:

```
doc1 = nlp(text1)
doc2 = nlp(text2)
doc1.similarity(doc2)
```

However, since we have already calculated the word vectors for each node description above, we will just use the cosine similarity built into `scikit-learn`. 

Note that we are creating now a complete node list since the nodes we are comparing might be either in the Barack or the Michelle Obama node lists...

In [15]:
def get_word_vec_similarity(node1, node2, node_ls):
    
    node1_vec = [tup[4] for tup in node_ls if tup[0] == node1]
    node2_vec = [tup[4] for tup in node_ls if tup[0] == node2]
    
    return cosine_similarity(node1_vec, node2_vec)

In [16]:
cs = get_word_vec_similarity('barack obama', 'president barack obama', full_dedup_node_tup_ls)
print(cs)

[[1.]]


## ...OK, that one was cheating...

...because the Google Knowledge Graph knew that these were the same thing.  The NER returned these as being different nodes, but Google knew better and gave the exact description for each.  This will not necessarily be the case in more complicated knowledge graphs.

In [17]:
cs = get_word_vec_similarity('barack obama', 'mitch mcconnell', full_dedup_node_tup_ls)
print(cs)

[[0.89986608]]


In [18]:
cs = get_word_vec_similarity('barack obama', 'hillary clinton', full_dedup_node_tup_ls)
print(cs)

[[0.94099334]]


In [19]:
cs = get_word_vec_similarity('barack obama', 'harvard law school', full_dedup_node_tup_ls)
print(cs)

[[0.89470368]]


In [20]:
cs = get_word_vec_similarity('barack obama', 'oman', full_dedup_node_tup_ls)
print(cs)

[[0.87748002]]


In [28]:
np.random.uniform(low=-1.0, high=1.0, size=(300,))

array([-0.66030315,  0.21017629, -0.96999948,  0.40863692, -0.0996688 ,
        0.62956923,  0.39212023, -0.45154716, -0.84673608,  0.07932166,
        0.9013387 ,  0.31209648,  0.428538  ,  0.43056846,  0.16249846,
       -0.14100601,  0.35965708, -0.81480086, -0.10669235, -0.50576842,
        0.68397926, -0.03217294, -0.43152521,  0.29264008,  0.56430447,
       -0.77589676,  0.78508795, -0.50276881, -0.95604656,  0.76775576,
        0.19381245,  0.89440471,  0.139772  ,  0.96216551, -0.95419435,
        0.2773476 ,  0.89445183, -0.70758328,  0.26642341,  0.15127036,
        0.92326232, -0.3139779 , -0.74181159,  0.24752297, -0.26524933,
        0.6426756 ,  0.32456654, -0.93813256, -0.44144314,  0.02897806,
       -0.54061467, -0.38581981,  0.9316153 ,  0.06010495,  0.39563128,
        0.4013969 , -0.38310475,  0.98971501, -0.76999374, -0.70067888,
       -0.43390566,  0.74649778, -0.19119958, -0.91318676,  0.73855285,
        0.62766323,  0.5080104 ,  0.65673224, -0.6017031 ,  0.32